# Examples

[**(quick-start)**](#quick-start) Adding a `@costly()` decorator to your function automatically adds the arguments `simulate: bool` and `cost_log: costly.Costlog` to it (and also `description: list[str]` which is useful for tracing the sources of costs in the breakdown).

By default, this uses [`LLM_Simulator_Faker.simulate_llm_call()`](costly/simulators/llm_simulator_faker.py) to simulate your function and [`LLM_API_Estimation.get_cost_real()`](costly/estimators/llm_api_estimation.py) as the estimator -- see [**(assumptions)**](#assumptions) for some brief comments on how these work by default. 

You can also use your own simulator and estimator, as shown in Example [**(customators)**](#customators). In particular you can subclass the existing simulator and estimator objects.

The default simulator and estimator assume that your function takes arguments `input_string: str, model: str, response_model: str | BaseModel)`, and supplies these arguments to the simulator and estimator. Regardless of which simulator and estimator you use, if your function does not take the same arguments as them (e.g. if your function adds a system prompt to `input_string`, or you use a different naming system for `model`s from that used in `LLM_API_Estimation.PRICES`), you can pass parameter mappings as shown in Example [**(param-mappings)**](#param-mappings). 

In particular see the example parameter mappings for [**(prompt-and-instructor)**](#prompt-and-instructor).

For more accurate cost tracking, you might want to calculate costs within the function, e.g. use info returned by the API call itself. For this, your function must response a `CostlyResponse` object as shown in Example [**(costly-response)**](#costly-response). In usage, your function will still return the normal output contained in `CostlyResponse.output`, and everything else will be passed to the simulator and estimator.

Finally, see [**(costlog-notes)**](#costlog-notes) for some notes on the `Costlog` class itself.

Oh and also see [**(invoice)**](#invoice) for how to use the `@invoice` decorator to functions to help `Costlog` track exactly what's using your money.

## quick-start

In [1]:
from openai import OpenAI
from costly import Costlog, costly


@costly()
def chatgpt(messages: dict, model: str) -> str:
    from openai import OpenAI

    client = OpenAI()
    response = client.chat.completions.create(
        model=model, messages=messages,
    )
    output_string = response.choices[0].message.content
    return output_string


cl = Costlog()
x = chatgpt(
    messages=[{"content": "Write the Lorem ipsum text", "role": "user"}],
    model="gpt-4o-mini",
    cost_log=cl,
    simulate=False,
)
y = chatgpt(
    messages=[{"content": "Write the Lorem ipsum text", "role": "user"}],
    model="gpt-4o-mini",
    cost_log=cl,
    simulate=True,
)
print(
    x,
    "\n---\n",
    y,
    "\n---\n",
    cl.totals,
    "\n---\n",
    cl.items[0],
    "\n---\n",
    cl.items[1],
)

c:\Users\abhim\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
c:\Users\abhim\Google Drive\Gittable\Code\costly\costly\estimators\llm_api_estimation.py:192: UserWarning: messages_to_input_tokens: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.
  warnings.warn(


Certainly! Here is the classic "Lorem Ipsum" text:

```
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
```

If you need a longer version or something specific, just let me know! 
---
 Cultural scientist community care that.
Imagine heavy she sing. Between goal available its then space final. Evidence another later stage.
Agency account necessary pressure. Key southern each war involve who.
Inside social training executive certainly close. Note old task size term condition loss. Claim chance crime type skill Congress book hair.
Hard oil place. Its example responsibility author measure still succ

Here's basically what's happening under the hood, courtesy of `costly.decorator.costly`:


In [2]:
from costly.simulators.llm_simulator_faker import LLM_Simulator_Faker
from costly.estimators.llm_api_estimation import LLM_API_Estimation

def _chatgpt(messages: list[dict[str, str]], model: str) -> str:
    from openai import OpenAI
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    output_string = response.choices[0].message.content
    return output_string

def chatgpt(messages: list[dict[str,str]], model: str, cost_log: Costlog=None, simulate: bool=False, description: list[str]=None) -> str:
    if simulate:
        return LLM_Simulator_Faker.simulate_llm_call(
            messages=messages,
            model=model,
            response_model=str,
            cost_log=cost_log,
            description=description,
        )
    if cost_log is not None:
        with cost_log.new_item() as (item, timer):
            output_string = _chatgpt(messages, model)
            cost_item = LLM_API_Estimation.get_cost_real(
                model=model,
                input_string=messages,
                output_string=output_string,
                description=description,
                timer=timer(),
            )
            item.update(cost_item)
    else:
        output_string = _chatgpt(input_string, model)
    return output_string


## assumptions

`LLM_Simulator_Faker`, when producing text, produces text of about `600 * 4.5` characters.

Generally we assume that 1 token is about 4.5 characters. Though actual token estimation does use `tiktoken` (unless you subclass `LLM_API_Estimation` and set `tokenize=_tokenize_rough`).

Generally we assume, for cost estimation, that output tokens are in the range `[0, 2048]`, and the min and max are computed accordingly. As a rule of thumb for complex projects the true value tends to be about 1/3 the way through, and for projects that receive quite short responses it would be much lower.

`LLM_API_Estimation.get_cost_real()` uses the `PRICES` dict to get the cost of a (real of simulated) API call, and to estimate time for a simulated API call. By default the standard OpenAI and Anthropic models are included; you can add more via subclassing.

All of this can be overriden by subclassing.

## customators

The defualt decorator behaviour is

```python
@costly(
    simulator=LLM_Simulator_Faker.simulate_llm_call,
    estimator=LLM_API_Estimation.get_cost_real,
)
```

These functions can be replaced by your own custom functions -- the best way to do this is probably to subclass the respective class.

[`costly.simulators.llm_simulator_faker`](costly/simulators/llm_simulator_faker.py) has some examples of how to subclass it. One obvious reason to subclass it is to have custom simulating functions for the types you are interested in. Although the default class "works" for any Pydantic basemodel etc., you might want to have a custom function -- e.g. if a value needs to be within a certain range, or if its distribution is not uniform, or if you want to use examples from your data, etc.

Again [`costly.estimators.llm_api_estimation`](costly/estimators/llm_api_estimation.py) has some examples of how to subclass it. The most obvious reason would be to add prices for other models we don't have listed (right now it's just OpenAI and Anthropic). The `PRICES` dict is like this:

```python
class LLM_API_Estimation:

    PRICES = {
        "gpt-4o": {
            "input_tokens": 5.0e-6,
            "output_tokens": 15.0e-6,
            "time": 18e-3,
        },
        "gpt-4o-mini": {
            "input_tokens": 0.15e-6,
            "output_tokens": 0.6e-6,
            "time": 9e-3,
        },
        ...
    }
```

Something like this would be quite natural:

```python
class My_Estimation(LLM_API_Estimation):
    PRICES = LLM_API_Estimation.PRICES | {"my_model": LLM_API_Estimation.PRICES["gpt-4o"]}
```

Note that `LLM_API_Estimation` _can_ handle things like `gpt-4o-2024-05-13` etc. because it `LLM_API_Estimation.get_model()` gets the longest prefix matching model name in `PRICES`. 


The default simulator and estimator have type signatures:

```python
class LLM_Simulator_Faker:

    @staticmethod
    def simulate_llm_call(
        input_string: str = None,  # must supply at least one of input_string, input_tokens, messages
        input_tokens: int = None,
        messages: list[dict[str, str]] = None,
        model: str = None,
        response_model: type = str,
        cost_log: Costlog = None,
        description: list[str] = None,
    ) -> str | Any:
        ...

class LLM_API_Estimation:

    @staticmethod
    def get_cost_real(
        model: str,
        input_tokens: int = None,
        output_tokens: int = None,
        input_string: str = None,
        messages: list[dict[str, str]] = None,
        output_string: str = None,
        timer: float = None,
        **kwargs,
    ) -> dict[str, float]:
        ...
```

## param-mappings

In [3]:
from costly import Costlog, costly
from costly.simulators.llm_simulator_faker import LLM_Simulator_Faker
from costly.estimators.llm_api_estimation import LLM_API_Estimation


@costly(
    messages=(lambda kwargs: [{"role": "user", "content": kwargs["prompt"]}]),
    model=(lambda kwargs: kwargs["model_name"]),
)
def chatgpt2(prompt: str, model_name: str) -> str:
    from openai import OpenAI

    client = OpenAI()
    response = client.chat.completions.create(
        model=model_name, messages=[{"role": "user", "content": prompt}]
    )
    output_string = response.choices[0].message.content
    return output_string


cl = Costlog()
chatgpt2(prompt="Hello", model_name="gpt-3.5-turbo", simulate=True, cost_log=cl)
chatgpt2(prompt="Hello", model_name="gpt-3.5-turbo", simulate=False, cost_log=cl)
print(cl.items[0],'\n---\n',cl.items[1])


c:\Users\abhim\Google Drive\Gittable\Code\costly\costly\estimators\llm_api_estimation.py:185: UserWarning: messages_to_input_tokens: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.
  warnings.warn(


{'cost_min': 4e-06, 'cost_max': 0.003076, 'time_min': 0.0, 'time_max': 73.728, 'input_tokens': 8, 'output_tokens_min': 0, 'output_tokens_max': 2048, 'calls': 1, 'model': 'gpt-3.5-turbo', 'simulated': True, 'input_string': None, 'messages': [{'role': 'user', 'content': 'Hello'}], 'output_string': None, 'description': None} 
---
 {'cost_min': 1.75e-05, 'cost_max': 1.75e-05, 'time_min': 0.5978679999825545, 'time_max': 0.5978679999825545, 'input_tokens': 8, 'output_tokens': 9, 'output_tokens_min': 9, 'output_tokens_max': 9, 'calls': 1, 'model': 'gpt-3.5-turbo', 'simulated': False, 'input_string': None, 'messages': [{'role': 'user', 'content': 'Hello'}], 'output_string': 'Hello! How can I help you today?', 'description': None}


Actually for the specific case of just remapping variables you can simply do:

In [4]:
from costly import Costlog, costly
from costly.simulators.llm_simulator_faker import LLM_Simulator_Faker
from costly.estimators.llm_api_estimation import LLM_API_Estimation


@costly(messages="message_history", model="model_name")
def chatgpt2(message_history: list[dict[str, str]], model_name: str) -> str:
    from openai import OpenAI

    client = OpenAI()
    response = client.chat.completions.create(
        model=model_name, messages=message_history
    )
    output_string = response.choices[0].message.content
    return output_string


cl = Costlog(
    totals_keys={
        "cost_min",
        "cost_max",
        "time_min",
        "time_max",
        "calls",
        "input_tokens",
        "output_tokens_min",
        "output_tokens_max",
    }
)
mh = [{"content": "Hello", "role": "user"}]
chatgpt2(message_history=mh, model_name="gpt-3.5-turbo", simulate=True, cost_log=cl)
chatgpt2(message_history=mh, model_name="gpt-3.5-turbo", simulate=False, cost_log=cl)
print(cl.items[0],'\n---\n',cl.items[1])


{'cost_min': 4e-06, 'cost_max': 0.003076, 'time_min': 0.0, 'time_max': 73.728, 'input_tokens': 8, 'output_tokens_min': 0, 'output_tokens_max': 2048, 'calls': 1, 'model': 'gpt-3.5-turbo', 'simulated': True, 'input_string': None, 'messages': [{'content': 'Hello', 'role': 'user'}], 'output_string': None, 'description': None} 
---
 {'cost_min': 1.75e-05, 'cost_max': 1.75e-05, 'time_min': 0.6519599999883212, 'time_max': 0.6519599999883212, 'input_tokens': 8, 'output_tokens': 9, 'output_tokens_min': 9, 'output_tokens_max': 9, 'calls': 1, 'model': 'gpt-3.5-turbo', 'simulated': False, 'input_string': None, 'messages': [{'content': 'Hello', 'role': 'user'}], 'output_string': 'Hello! How can I assist you today?', 'description': None}


## prompt-and-instructor

More examples of parameter mappings.

In [9]:
from costly import Costlog, costly
from costly.simulators.llm_simulator_faker import LLM_Simulator_Faker
from costly.estimators.llm_api_estimation import LLM_API_Estimation


@costly(
    input_tokens=lambda kwargs: LLM_API_Estimation.prompt_to_input_tokens(**kwargs),
)
def chatgpt_prompt(
    prompt: str, model: str, system_prompt: str = "You are a helpful assistant."
) -> str:
    from openai import OpenAI

    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"content": prompt, "role": "user"},
            {"content": system_prompt, "role": "system"},
        ],
    )
    output_string = response.choices[0].message.content
    return output_string


cl = Costlog()
chatgpt_prompt(
    prompt="Hey",
    model="gpt-4o-mini",
    simulate=True,
    cost_log=cl,
)
chatgpt_prompt(
    prompt="Hey",
    model="gpt-4o-mini",
    simulate=False,
    cost_log=cl,
)
print(cl.items[0], "\n---\n", cl.items[1])

c:\Users\abhim\Google Drive\Gittable\Code\costly\costly\estimators\llm_api_estimation.py:192: UserWarning: messages_to_input_tokens: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.
  warnings.warn(


{'cost_min': 2.7e-06, 'cost_max': 0.0012315, 'time_min': 0.0, 'time_max': 18.432, 'input_tokens': 18, 'output_tokens_min': 0, 'output_tokens_max': 2048, 'calls': 1, 'model': 'gpt-4o-mini', 'simulated': True, 'input_string': None, 'messages': None, 'output_string': None, 'description': None} 
---
 {'cost_min': 8.1e-06, 'cost_max': 8.1e-06, 'time_min': 0.8648557999986224, 'time_max': 0.8648557999986224, 'input_tokens': 18, 'output_tokens': 9, 'output_tokens_min': 9, 'output_tokens_max': 9, 'calls': 1, 'model': 'gpt-4o-mini', 'simulated': False, 'input_string': None, 'messages': None, 'output_string': 'Hello! How can I assist you today?', 'description': None}


In [6]:
import instructor
from pydantic import BaseModel
from openai import OpenAI
from instructor import Instructor
from costly import costly, Costlog
from costly.estimators.llm_api_estimation import LLM_API_Estimation


@costly(
    input_string=lambda kwargs: LLM_API_Estimation._get_raw_messages_instructor(**kwargs),
)
def chatgpt_instructor(
    messages: list[dict[str, str]],
    model: str,
    client: Instructor,
    response_model: BaseModel,
) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_model=response_model,
    )
    return response


class PersonInfo(BaseModel):
    name: str
    age: int


cl = Costlog()
chatgpt_instructor(
    messages={"content": "Hey", "role": "user"},
    model="gpt-3.5-turbo",
    response_model=PersonInfo,
    client=instructor.from_openai(OpenAI()),
    simulate=True,
    cost_log=cl,
)
chatgpt_instructor(
    messages={"content": "Hey", "role": "user"},
    model="gpt-3.5-turbo",
    response_model=PersonInfo,
    client=instructor.from_openai(OpenAI()),
    simulate=False,
    cost_log=cl,
)
print(cl.items[0], '\n---\n', cl.items[1])

2024-09-05 23:00:11,977 DEBUG instructor: Instructor Request: mode.value='tool_call', response_model=<class '__main__.PersonInfo'>, new_kwargs={'messages': [{'content': 'Hey', 'role': 'user'}], 'model': 'gpt-3.5-turbo', 'tools': [{'type': 'function', 'function': {'name': 'PersonInfo', 'description': 'Correctly extracted `PersonInfo` with all the required parameters with correct types', 'parameters': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}}, 'required': ['age', 'name'], 'type': 'object'}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'PersonInfo'}}}
2024-09-05 23:00:11,992 DEBUG instructor: max_retries: 3
2024-09-05 23:00:12,015 DEBUG instructor: Patching `client.chat.completions.create` with mode=<Mode.TOOLS: 'tool_call'>
2024-09-05 23:00:12,015 DEBUG instructor: Instructor Request: mode.value='tool_call', response_model=<class '__main__.PersonInfo'>, new_kwargs={'messages': [{'content': 'Hey', 'role': 'user'

{'cost_min': 1.7e-05, 'cost_max': 0.003089, 'time_min': 0.0, 'time_max': 73.728, 'input_tokens': 34, 'output_tokens_min': 0, 'output_tokens_max': 2048, 'calls': 1, 'model': 'gpt-3.5-turbo', 'simulated': True, 'input_string': "HeyPersonInfoCorrectly extracted `PersonInfo` with all the required parameters with correct typesdict_values(['Name', 'string'])dict_values(['Age', 'integer'])", 'messages': 'Hey', 'output_string': None, 'description': None} 
---
 {'cost_min': 1.7e-05, 'cost_max': 1.7e-05, 'time_min': 0.6298766999971122, 'time_max': 0.6298766999971122, 'input_tokens': 34, 'output_tokens': 0, 'output_tokens_min': 0, 'output_tokens_max': 0, 'calls': 1, 'model': 'gpt-3.5-turbo', 'simulated': False, 'input_string': "HeyPersonInfoCorrectly extracted `PersonInfo` with all the required parameters with correct typesdict_values(['Name', 'string'])dict_values(['Age', 'integer'])", 'messages': 'Hey', 'output_string': PersonInfo(name='Alice', age=30), 'description': None}


## costly-response

For more accurate estimation of _real_ (not simulated) costs, we might want to calculate costs within the function.

To do this, your function must return a `CostlyResponse` object.

```python
@dataclass
class CostlyResponse:
    output: Any
    cost_info: dict[str, Any]
```

 In actual usage, your function will only return the `output` field -- the `@costly` decorator will take care of this.


In [7]:
from costly import Costlog, costly, CostlyResponse


@costly()
def chatgpt(messages: list[dict[str, str]], model: str) -> str:
    from openai import OpenAI

    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    return CostlyResponse(
        output=response.choices[0].message.content,
        cost_info={
            "input_tokens": response.usage.prompt_tokens,
            "output_tokens": response.usage.completion_tokens,
        },
    )


cl = Costlog()
x = chatgpt(
    messages=[{"content": "Write the Lorem ipsum text", "role": "user"}],
    model="gpt-4",
    cost_log=cl,
    simulate=True,
)
y = chatgpt(
    messages=[{"content": "Write the Lorem ipsum text", "role": "user"}],
    model="gpt-4",
    cost_log=cl,
    simulate=False,
)
print(x, "\n---\n", y, "\n---\n", cl.items[0], "\n---\n", cl.items[1])

c:\Users\abhim\Google Drive\Gittable\Code\costly\costly\estimators\llm_api_estimation.py:192: UserWarning: messages_to_input_tokens: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.
  warnings.warn(


Language set out air. Alone sense physical agent national spend. Form school out benefit represent.
Action concern reach direction. Alone image culture along.
Each head deal officer environment time east. Near drug economic his claim put.
Church participant star lay fish score.
Brother parent want first. However that writer visit do often. Over left former control myself audience.
Fine father seem professional note you. Begin thousand approach authority he bit only.
Both young itself nation scene western here easy. Couple decision law management scene. Customer mean fast.
Million television voice toward there specific. Dog improve cold they us. Research sport expect it bill.
Only debate sport join word them want.
Staff although effect model significant skill pattern area. Always like resource certainly office another.
Remain mention high easy stuff movement themselves. Piece newspaper nice grow. Spring girl approach yet century two force. Nice address wife early pass.
Book scientist of

In [2]:
import instructor
from pydantic import BaseModel
from instructor import Instructor
from typing import Any
from openai import OpenAI
from costly import Costlog, costly, CostlyResponse
from costly.estimators.llm_api_estimation import LLM_API_Estimation


@costly(
    input_tokens=lambda kwargs: LLM_API_Estimation.get_input_tokens_instructor(
        **kwargs
    ),
)
def chatgpt_instructor(
    messages: list[dict[str, str]],
    model: str,
    client: Instructor,
    response_model: BaseModel,
) -> str:
    response = client.chat.completions.create_with_completion(
        model=model,
        messages=messages,
        response_model=response_model,
    )
    # print(
    #     LLM_API_Estimation._get_raw_messages_instructor(
    #         messages=messages,
    #         model=model,
    #         client=client,
    #         response_model=response_model,
    #         process=False,
    #     )
    # )
    output_string, cost_info = response
    return CostlyResponse(
        output=output_string,
        cost_info={
            "input_tokens": cost_info.usage.prompt_tokens,
            "output_tokens": cost_info.usage.completion_tokens,
        },
    )


class PersonInfo(BaseModel):
    name: str
    age: int

class FOOMODEL(BaseModel):
    name: str
    age: int
    bmi: float
    metadata: dict[str, Any] | None = None


class BARMODEL(BaseModel):
    foo: FOOMODEL
    fookids: list[FOOMODEL]

cl = Costlog()
x = chatgpt_instructor(
    messages=[{"content": "Hey", "role": "user"}],
    model="gpt-4",
    response_model=FOOMODEL,
    client=instructor.from_openai(OpenAI()),
    simulate=True,
    cost_log=cl,
)
y = chatgpt_instructor(
    messages=[{"content": "Hey", "role": "user"}],
    model="gpt-4",
    response_model=FOOMODEL,
    client=instructor.from_openai(OpenAI()),
    simulate=False,
    cost_log=cl,
)
print(x, "\n---\n", y, "\n---\n", cl.items[0], "\n---\n", cl.items[1])

2024-09-06 19:06:24,864 DEBUG instructor: Patching `client.chat.completions.create` with mode=<Mode.TOOLS: 'tool_call'>


2024-09-06 19:06:24,867 DEBUG instructor: Instructor Request: mode.value='tool_call', response_model=<class '__main__.FOOMODEL'>, new_kwargs={'messages': [{'content': 'Hey', 'role': 'user'}], 'model': 'gpt-4', 'tools': [{'type': 'function', 'function': {'name': 'FOOMODEL', 'description': 'Correctly extracted `FOOMODEL` with all the required parameters with correct types', 'parameters': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'bmi': {'title': 'Bmi', 'type': 'number'}, 'metadata': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'title': 'Metadata'}}, 'required': ['age', 'bmi', 'name'], 'type': 'object'}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'FOOMODEL'}}}
2024-09-06 19:06:24,875 DEBUG instructor: max_retries: 3
c:\Users\abhim\Google Drive\Gittable\Code\costly\costly\estimators\llm_api_estimation.py:287: UserWarning: Field title not found in tokenization function
  warnings.warn(
c:\U

mode.value='tool_call', response_model=<class '__main__.FOOMODEL'>, new_kwargs={'messages': [{'content': 'Hey', 'role': 'user'}], 'model': 'gpt-4', 'tools': [{'type': 'function', 'function': {'name': 'FOOMODEL', 'description': 'Correctly extracted `FOOMODEL` with all the required parameters with correct types', 'parameters': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'bmi': {'title': 'Bmi', 'type': 'number'}, 'metadata': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'title': 'Metadata'}}, 'required': ['age', 'bmi', 'name'], 'type': 'object'}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'FOOMODEL'}}}
name='Beautiful successful family Mr. Its yourself scientist.\nArt attack phone walk. Office factor civil try appear single.\nCourse southern media local among. Everyone hear Republican white cold yourself perhaps after. Professional away heavy yes kid certain throughout.\nAmong story race the

## costlog-notes

The default [`costly.Costlog`](costly/costlog.py) class has two modes: `memory` and `jsonl`. The default is `memory`, but for large projects you may want to use `jsonl`: this dumps the cost log into a `.costly` folder in your working directory.

The other thing that can be customized is the `totals_keys` parameter, which is a set of keys to aggregate costs by. By default it is `{"cost_min", "cost_max", "time_min", "time_max", "calls"}`, i.e. it tracks the range of possible costs and running times (`max` and `min` are usually only different when simulating because then you have to estimate). Out-of-the box you can customize it to also track `input_tokens`, `output_tokens_min`, `output_tokens_max`; any other customizations will only make sense if you are using your own estimator.

You will want to change `totals_keys` if you want to use this package for things other than LLM costs, or if you want to track something other than `min` and `max`, e.g. some estimate of the average, or percentiles or whatever.

## invoice

While the `@costly()` decorator goes on your "load-bearing" functions (e.g. your LLM wrapper), the `@invoice()` decorator goes on your "high-level" functions whose usage you want to be reported in your final `Costlog` to see a breakdown. This adds a `description` argument to your function, and as long as this argument is passed unbroken to your load-bearing functions (e.g. via `**kwargs`), you Costlog items will contain a `description` field tracing where they came from.

In [2]:
from costly import invoice

@invoice()
def manufacture_cups(**kwargs):
    manufacture_steel(**kwargs)
    educate_potters(**kwargs)
    return

@invoice("educating potters")
def educate_potters(**kwargs):
    capture_enemies(**kwargs)
    return

@invoice("manufacturing steel")
def manufacture_steel(**kwargs):
    mine_iron(**kwargs)
    return

@invoice("mining iron")
def mine_iron(**kwargs):
    print(kwargs["description"])
    return

@invoice("capturing enemies")
def capture_enemies(**kwargs):
    print(kwargs["description"])
    return

manufacture_cups()
### prints:
# [{'func': 'manufacture_cups', 'args': [], 'kwargs': {}}, 'manufacturing steel', 'mining iron']
# [{'func': 'manufacture_cups', 'args': [], 'kwargs': {}}, 'educating potters', 'capturing enemies']


[{'func': 'manufacture_cups', 'args': [], 'kwargs': {}}, 'manufacturing steel', 'mining iron']
[{'func': 'manufacture_cups', 'args': [], 'kwargs': {}}, 'educating potters', 'capturing enemies']


If description is not provided, it will default to the function's name and arguments like above.

`@invoice` uses the more general decorator [`@updatable_default`](costly/updatable_default.py); see the documentation therein it's kinda cool.